<a href="https://colab.research.google.com/github/JEEVA-JOSLIN/DEEP-LEARNING-ASSIGNMENT/blob/main/DEEP_LEARNING_ASSIGNMENT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('Tweets.csv')

In [ ]:
print(df.head())

             tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnar

In [ ]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

In [ ]:
df['cleaned_tweet'] = df['text'].apply(preprocess_text)

In [ ]:
model = Word2Vec(sentences=df['cleaned_tweet'], vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
print(model.wv.most_similar('good'))

[('response', 0.9949643611907959), ('haha', 0.9949449300765991), ('lol', 0.9938860535621643), ('appreciate', 0.99386066198349), ('love', 0.9938084483146667), ('oh', 0.9933992624282837), ('american', 0.9933134317398071), ('job', 0.9927789568901062), ('example', 0.9927170276641846), ('definitely', 0.9927154183387756)]


In [ ]:
def get_vector(tweet):
    word_vectors = []
    for word in tweet:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

In [ ]:
df['word2vec_features'] = df['cleaned_tweet'].apply(get_vector)

In [ ]:
X = np.array(df['word2vec_features'].to_list())
y = df['airline_sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 2}).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7438524590163934
              precision    recall  f1-score   support

           0       0.78      0.92      0.85      1889
           1       0.64      0.52      0.57       459
           2       0.60      0.36      0.45       580

    accuracy                           0.74      2928
   macro avg       0.67      0.60      0.62      2928
weighted avg       0.73      0.74      0.72      2928

